##### [< Forrige](5%20-%20profittmaksimering.ipynb)     |     [Neste >](6%20-%20logikk-filtrering-betingelser.ipynb)

# 6 - Oppslag og pandas

Pandas er den pakken de  fleste bruker til å importere og behandle data. Det første du må gjøre for å bruke pandas er selvsagt å hente data. Dette kan gjøres på flere måter. Som vi husker fra forelesning 3 - matplotlib, gan vi hente inn  en csv-fil (kommadelt) slik:

#### Eksempel 1:

In [ ]:
pd.read_csv('./data/schooling-gdp.csv')

## Titlon

En annen måte å gjøre det på er via en dataserver. På [https://titlon.uit.no/](https://titlon.uit.no/) ligger det børsinformasjon fra Oslo Børs. For å hente data fra Titlon gjør du følgende:

1. gå inn på [https://titlon.uit.no/](https://titlon.uit.no/) 
2. trykk på "Python"
3. logg inn (om du får en feilmelding etter innlogging følger du bare lenken over på nytt)
4. Kopierer teksten i tekstboksen
5. lim den inn under


#### Eksempel 1:

In [ ]:
#Lim inn teksten fra Titlon her:
import pymysql
#you may substitute pymysql for pymssql if you prefer the MS SQL client
con = pymysql.connect(host='titlon.uit.no', 
		user="esi000@uit.no", 
		password="vm@z3qIdBRJ6EvqDOP8S4", 
		database='OSE')  

sql="SELECT * FROM equity WHERE Name='Equinor' ORDER BY `Date`"
df=pd.read_sql_query(sql, con)
df



Koden leser inn dataene til en pandas `dataframe`, eller dataramme. Denne er kalt `df` her. Om vi ser nærmere på koden, så ser vi at den gjør følgende:

1. lager et objekt `con` som representerer forbindelsen med databasen. Denne lages ved å bruke `connect()`-funksjonen til `pymysql`, som er en pakke for å hente data fra MySQL-databaser. Input er navn på server, brukernavn, passord (automatisk generert) og navn på database. 

2. lager en streng `sql` med kommandoen til databasen, i SQL-språk
3. lager en `dataframe` som kalles `df` ved å bruke  `read_sql_query`-funksjonen i pandas

Før vi fortsetter er det kanskje greit å forklare hva SQL er. SQL er et språk laget spesifikt for å hente ut data fra dataabaser. Nesten alle databaser er SQL-basert. Syntaksen kan virke litt knotete, men kan være lurt å lære seg dette språket. En god SQL-spørring kan spare deg for mye programmering senere. Dette er imidlertid ikke et kurs i SQL, så vi går ikke nærmere inn på denne syntaksen her. 

I tabellen vi henter, over, mangler endel variabler fordi det ikke er plass. For å se navnet på alle variablene kan du bruke `keys()`-funksjonen til datarammen:

In [ ]:
df.keys()

En dataframe ligner litt på en oppslagsliste. Du henter frem variabelen du vil bruke ved hjelp av hakeparenteser. I Titlondatabasen er avkastningen `lnDeltaP`, så la oss ta en kikk på på den:

In [ ]:
df['lnDeltaP']

Det viser seg at i aksjemarkedet, så har aksjene en tendens til å bevege seg i samme retning. Går børsen først ned, så vil det gjelde de aller fleste aksjene på børsen. Gjennomsnittetlig avkastning for alle børsens aksjer måles av *børsindeksen*. I Titlon er børsindeksen lagt inn som en variabel for hvert selskap under navnet `lnDeltaOSEBX`. La oss derfor undersøke om det stemmer at det er en nær sammenheng mellom Equinors kurs og børsindeksen, ved hjelp av `matplotlib`:

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(df['lnDeltaP'],df['lnDeltaOSEBX'])

Du kan finne informasjon om andre aksjer enn Equinor om du vil. Koden under kjører en SQL-spørring som henter navnene på alle selskap i databasen etter 2015. Om du vil hente ut et annet 

In [ ]:
sql="SELECT distinct Name,ISIN,SecurityId,CompanyId FROM equity WHERE Year(Date)>2015 ORDER BY `Name`"
companies=pd.read_sql_query(sql, con)
list(companies.values)

In [ ]:
sql="SELECT distinct ISIN,date, lnDeltaOSEBX,lnDeltaP FROM equity WHERE Name='Equinor' or Name='Adevinta' or Name='SalMar' ORDER BY `Date`"
df=pd.read_sql_query(sql, con)

In [ ]:
df=df.sort_values(['ISIN','date'])
panel=df.groupby('ISIN')
panel

## Eurostat

Vi kan sjekke hvilke tabeller eurostat har slik

In [ ]:
import eurostat
toc = eurostat.get_toc_df()
gdp_title=toc[['GDP per capita' in i for i in toc['title']]]
gdp_title

La oss se på tabellen "Purchasing power adjusted GDP per capita", som har kode 'sdg_10_10'. Da bruker vi funksjonen `get_data_df`. 

In [ ]:
gdp_data = eurostat.get_data_df('sdg_10_10')
gdp_data

Vi kan nå plott en graf. For å gjøre det mer oversiktlig, tar vi kun med radene der kolonnen 'na_item' er lik 'EXP_PPS_EU27_2020_HAB', og vi fjerner alle EU-grupperingene, uten EU28. 

Siden det Angela Merkel akkurat er gått av, ser vi på utviklingen i BNP justert for kjøpekraft til Tyskland (DE), Norge og EU siden Merkel tiltrådte i 2005 spesielt. Det kan vi gjøre ved å duse ut alle andre tidsserier (alpha=0.3) og forsterke de nevnte. 

Vi "rebaserer" så tallene slik at alle starter på 100 med `gdp_np=100*gdp_np/gdp_np[:,0:1]`.

Da får vi denne grafen:

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
gdp_data = eurostat.get_data_df('sdg_10_10')
gdp_data = gdp_data[gdp_data['na_item']=='EXP_PPS_EU27_2020_HAB']
gdp_data = gdp_data[~gdp_data['geo\\time'].isin(['EA18','EA19','EU27_2007','EU27_2020','EA18','EA18'])]
years=gdp_data.keys()[9:]
country_codes=list(np.array(gdp_data)[:,3])
gdp_np=np.array(gdp_data)[:,9:]

#rebasing the gdp's:
gdp_np=100*gdp_np/gdp_np[:,0:1]

plt.figure(figsize=(20,10))
p=plt.plot(years,gdp_np.T,alpha=0.3)
p[country_codes.index('DE')].set_linewidth(5)
p[country_codes.index('NO')].set_linewidth(5)
p[country_codes.index('EU28')].set_linewidth(5)
p[country_codes.index('DE')].set_alpha(1)
p[country_codes.index('NO')].set_alpha(1)
p[country_codes.index('EU28')].set_alpha(1)
plt.ylim([90, 200])
plt.legend(p,country_codes,loc='upper left',ncol=4)

## Oppgaver:

1. Hent ut to andre selskap fra titlon, og plott dem med `scatter()`-funksjonen for å se om det er en sammenheng mellom avkastningene.  

##### [< Forrige](5%20-%20profittmaksimering.ipynb)     |     [Neste >](6%20-%20logikk-filtrering-betingelser.ipynb)